In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.interpolate import griddata
import ast
import os

# 设置目标时间和城市
target_dt = datetime.strptime("2018-09-21", "%Y-%m-%d")
city = "shanghai"
day_of_year = target_dt.timetuple().tm_yday

In [14]:
# 读取数据
df = pd.read_csv(f"{city}_trims_with_simulations.csv")
df = df[df['date'] == f'{day_of_year}D'].copy()
df['lat'] = df['latitude'] / 10000
df['lon'] = df['longitude'] / 10000

# 解析字符串为列表
df['sim_values'] = df['TRIMS_LST_SIM'].apply(lambda x: np.array(ast.literal_eval(x)))
# 计算偏移
df['offset'] = df['TRIMS_LST'] - df['sim_values'].apply(lambda x: np.max(x))
# 应用偏移修正
df['adjusted_sims'] = df.apply(lambda row: row['sim_values'] + row['offset'], axis=1)

# 区域边界
if city == 'beijing':
    lon_l, lon_r, lat_b, lat_u = 115.43, 117.52, 39.44, 41.05
elif city == 'shanghai':
    lon_l, lon_r, lat_b, lat_u = 120.836619, 122.165824, 30.631414, 31.7
elif city == 'shenzhen':
    lon_l, lon_r, lat_b, lat_u = 113.70, 114.70, 22.45, 22.85

# 网格设置
GRID_SIZE = 50  # 米
earth_radius = 6378137.0
pi = np.pi
meter_per_degree = earth_radius * pi / 180.0
lat_step = GRID_SIZE / meter_per_degree
ratio = np.cos((lat_b + lat_u) * pi / 360)
lon_step = lat_step / ratio

grid_lat = np.arange(lat_b, lat_u, lat_step)
grid_lon = np.arange(lon_l, lon_r, lon_step)
grid_x, grid_y = np.meshgrid(grid_lon, grid_lat)

H, W = grid_x.shape
T = 12  # 12小时

temperature_cube = np.full((T, H, W), np.nan)

In [15]:
# 插值并填充12个时间步
for t in range(T):
    values = df['adjusted_sims'].apply(lambda x: x[t]).values
    points = df[['lon', 'lat']].values
    grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

    # 保真处理
    for idx, row in df.iterrows():
        i = int((row['lat'] - lat_b) / lat_step)
        j = int((row['lon'] - lon_l) / lon_step)
        if 0 <= i < H and 0 <= j < W:
            grid_z[i, j] = row['adjusted_sims'][t]
    
    temperature_cube[t] = grid_z

# 保存为npy
output_path = f'./tmp_sim/tmp_sim_{city}_{target_dt.strftime("%Y%m%d")}_cube.npy'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
np.save(output_path, temperature_cube)
print(f"Saved temperature grid to {output_path}")

Saved temperature grid to ./tmp_sim/tmp_sim_shanghai_20180921_cube.npy
